# Entrainement :  jeux de données "économétrie" de Régis BOURDONNAIS

https://regisbourbonnais.dauphine.fr/fr/publications/econometrie-manuel-et-exercices-corriges.html

Je possède la 11e édition, données récupérées pour la 9e, pas envie de travailler sur Stata :/  

In [1]:
import pandas as pd
import numpy as np
import statsmodels as sm

#  Exercice C5EX5 page 172 : Test d'éxogénéité de Haussman (deux versions) 

Haussman's test for exogeneity allows for the detection of a possible correlation between the error term $\epsilon_i$ and one or multiple explicative variables $X_i$ :: $E[\epsilon_i X_i] \ne 0$

$H_0 : Cov(X_i,\epsilon_i) = 0$ contre l'alternative $H_1 : Cov(X_i,\epsilon_i) \ne 0$

Under the null hypothesis, OLSE and IVE converge to the same value, otherwise OLSE is biased and does not converge.
Basically : if the ull hypothesis is accepted (or rather not rejected), we believe OLSE and IVE to converge to the same value, meaning no correlation between the error term and endogenous variables. No need resort to IVs. 

## 1 - Méthode de regression sur modèle augmenté : 

In [2]:
df=pd.read_excel(r"C:\Users\hugue\Documents\Université\Master Statistiks\WS21\Econometrical methods\Entrainement code\all_data_regis_bourdonais\C5EX5.XLS")
# "r" au debut pour convertir le path en string simple : plus de problème d'encodage. Parfois remplacer les / par des \ peut aussi résoudre le problème

Méthode de la "régression augmentée" : 

1- Commencer par faire la regression des / de la variable endogène par les variables instumentales

2- Estimation de la variable endogène et de ses coefficients par les MCO

3- Variable instrumentales estimées dans le modèle initale

4- Test de significance des coefficients des variables instrumentales : $H_0  : \gamma = 0$

In [3]:
df

,Y,X,Z
0,15.30,17.30,3.0
1,19.91,21.91,7.0
2,20.94,22.96,5.4
3,19.66,21.86,9.8
4,21.32,23.72,7.8
5,18.33,20.73,7.8
6,19.59,22.19,10.2
7,21.30,23.90,10.2
8,20.93,23.73,12.6
9,21.64,24.44,12.6


In [4]:
#regression de X sur Z : 
#statsmodels pour linear regression 

X,Y,Z = df['X'],df['Y'],df['Z']

from statsmodels.api import add_constant as add_constant
from statsmodels.api import OLS as OLS
#Il faut importer le module pour les constantes directement depuis l'api sinon avec sm.tools.tools tu vas pas aller loin
Z = add_constant(Z,prepend = True,has_constant = 'add')

#Regression of endogene variable X on instrumental variable Z : 
X_hat = OLS(X,Z).fit()
regression_1 = X_hat.summary()
print(regression_1)

#Calcul nouvelle série ajustée : 
X_hat = [18.3725 + 0.4407*df['Z'][i] for i in range(df.index.stop)]
df['X_hat'] = X_hat
print(df)

                            OLS Regression Results                            
Dep. Variable:                      X   R-squared:                       0.871
Model:                            OLS   Adj. R-squared:                  0.864
Method:                 Least Squares   F-statistic:                     121.9
Date:                Tue, 15 Mar 2022   Prob (F-statistic):           1.90e-09
Time:                        13:22:52   Log-Likelihood:                -30.018
No. Observations:                  20   AIC:                             64.04
Df Residuals:                      18   BIC:                             66.03
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         18.3725      0.622     29.514      0.0

In [18]:
# Definition du modèle avancé pour la regression : on ajoute la variable estimé X_hat :
# Nouveau vecteur des regresseurs : X_aug = (X,X_hat)

X_aug = add_constant(df[['X','X_hat']])

Y_hat = OLS(Y,X_aug).fit()
regression_2 = Y_hat.summary()
print(regression_2)


                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                     5322.
Date:                Tue, 15 Mar 2022   Prob (F-statistic):           1.67e-24
Time:                        11:59:22   Log-Likelihood:                 17.691
No. Observations:                  20   AIC:                            -29.38
Df Residuals:                      17   BIC:                            -26.40
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.1534      0.213     10.123      0.0

### Interprétation : 

$X_{aug}$ qui contient $ X^{hat} $, la variable qui prend en compte les variations de $Z_i$ sur $X_i$, est significativement supérieur à zéro. C'est donc un coefficient justifiable, et une partie de la variation est endogène et expliquée par $Z_i$ au travers de $X^{hat}$. 

## 2 - Test de différence (Statistique classique)

Relies on the following statistic : 

$H = (\beta\hat_{IV} - \beta\hat_{OLS})'[Var(\beta\hat_{IV}) - V(\beta\hat_{OLS})]^-1(\beta\hat_{IV} - \beta\hat_{OLS})$

and $H$ ~ $X^{2, 1-\alpha}_{k}$    for an $\alpha$ level of confidence given